## Initialization

### Importing packages

In [1]:
# System packages
import sys
import time
import warnings
import os
import fiona
from collections import Counter

# Non-geo numeric packages
import numpy as np
import math
from itertools import product, combinations
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Network and OSM packages
import networkx as nx
import osmnx as ox
city_geo = ox.geocoder.geocode_to_gdf

# Earth engine packages
import ee
import geemap

# General geo-packages
from rasterstats import zonal_stats
import pyproj
from pyproj import CRS
import libpysal
import rasterio
import rioxarray
import geopandas as gpd
import shapely
from shapely import geometry
from shapely.ops import transform
from shapely.geometry import Point, MultiLineString, LineString, Polygon, MultiPolygon

# Multiprocessing packages
import multiprocessing
from joblib import Parallel, delayed

c:\Users\Gebruiker\anaconda3\envs\ssml\lib\site-packages\osmnx\projection.py:3: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  import geopandas as gpd


### Importing and initializing data
- Exported data from the previous notebook is imported.
- The respective urban areas' EPSG and area are defined.

In [4]:
# Import population grid of respective urban area
popgrid = gpd.read_file('popgridmanchester.gpkg')

In [8]:
popgrid

,dissolve_key,row3,col3,population,grid_lon,grid_lat,grid_lon_4326,grid_lat_4326,buffer_population,buffer_area,buffer_pop_density,geometry,centroid
0,10-621,10,621,0,390427.904831,420645.529660,-2.146435,53.682200,1.711105,130889.763169,0.000013,"POLYGON ((390417.254 420647.897, 390433.850 42...",POINT (-2.14643 53.68220)
1,10-628,10,628,0,390871.732026,420671.933032,-2.139770,53.682474,2.195006,352519.406529,0.000006,"POLYGON ((390837.929 420707.144, 390893.725 42...",POINT (-2.13977 53.68247)
2,10-629,10,629,0,390925.124111,420675.744940,-2.138872,53.682475,2.092065,372651.203601,0.000006,"POLYGON ((390893.725 420707.144, 390956.523 42...",POINT (-2.13887 53.68248)
3,10-630,10,630,0,390987.922453,420675.744940,-2.137921,53.682476,2.251067,403440.920467,0.000006,"POLYGON ((390956.523 420707.144, 391019.322 42...",POINT (-2.13792 53.68248)
4,10-631,10,631,0,391050.720795,420675.744940,-2.136971,53.682478,2.395421,434667.971694,0.000006,"POLYGON ((391019.322 420707.144, 391082.120 42...",POINT (-2.13697 53.68248)
...,...,...,...,...,...,...,...,...,...,...,...,...,...
324747,99-740,99,740,0,397895.740085,415086.692492,-2.033295,53.632316,30.625313,616211.171034,0.000050,"POLYGON ((397864.341 415118.092, 397927.139 41...",POINT (-2.03329 53.63232)
324748,99-741,99,741,0,397958.538427,415086.692492,-2.032345,53.632316,29.462864,555520.656428,0.000053,"POLYGON ((397927.139 415118.092, 397989.938 41...",POINT (-2.03234 53.63232)
324749,99-742,99,742,0,398021.336769,415086.692492,-2.031395,53.632316,27.925428,493591.775958,0.000057,"POLYGON ((397989.938 415118.092, 398052.736 41...",POINT (-2.03140 53.63232)
324750,99-743,99,743,0,398084.135111,415086.692492,-2.030446,53.632317,25.495636,431314.046552,0.000059,"POLYGON ((398052.736 415118.092, 398115.534 41...",POINT (-2.03045 53.63232)


In [6]:
# Define local parameters for the EPSG and area for respective urban area
epsg = "EPSG:27700"
area = "Greater Manchester"

## Distance Calculations
- The street network of the urban area is either loaded or downloaded for calculating the distances.
- The distances to nearest nodes from each grid centroid is calculated.
- All points of interest (POIs) are extracted from OpenStreetMap (OSMNX).
- The distances from each grid to the nearest POI is calculated using the nearest nodes from both the grid and POIs.

### Either load or download street network of the respective urban area

In [10]:
# Optional: Load in street network graph for the respective urban area if local file is already available
Gproj = ox.load_graphml('network.graphml')

In [ ]:
# Otherwise: Download street network of the urban area from OSMNX and project to local EPSG
G = ox.graph_from_place(area, network_type='all', simplify=True)
Gproj = ox.project_graph(G, to_crs=epsg)

# Save to graphml
ox.io.save_graphml(Gproj, 'network.graphml')

### Calculate the nearest node to each grid centroid with a threshold of 1000 meters

In [11]:
# List of grid centroid nearest nodes and their distances
nearest_nodes, dist = ox.distance.nearest_nodes(Gproj, popgrid['grid_lon'],
                                                popgrid['grid_lat'], return_dist=True)

# Apply vectorization to get the nearest grid nodes with a threshold
mask = np.vectorize(isinstance)(nearest_nodes, int)
nearest_nodes_g = np.where(mask, nearest_nodes, np.nan)
dist_g = np.where(mask, dist, np.nan)
valid_nodes_grid = np.array([nearest_nodes_g, dist_g], np.float64)
valid_nodes_grid = np.where(valid_nodes_grid[1]<1000, valid_nodes_grid[0], np.nan)

### Obtain the POIs from OpenStreetMap (OSMnx)

2.3.1 Bus Stops

In [12]:
# Extracting bus stops
bus_stops = ox.geometries_from_place(area, tags={'highway': 'bus_stop'})
bus_stops = bus_stops.to_crs(epsg)
bus_stops = bus_stops[['geometry']]
bus_stops = bus_stops.reset_index()

Train Stations

In [10]:
# Extracting train stations
train_stations = ox.geometries_from_place(area, tags={'railway': 'station'})
train_stations = train_stations.to_crs(epsg)
train_stations = train_stations[['geometry']]
train_stations = train_stations.reset_index()

Restaurants

In [18]:
# Extracting restaurants
restaurants = ox.geometries_from_place(area, tags={'amenity': ['bar', 'pub', 'restaurant', 'cafe']})
restaurants = restaurants.to_crs(epsg)
restaurants = restaurants[['geometry']]
restaurants = restaurants.reset_index()

Fast Food

In [9]:
# Extracting fast food
fast_food = ox.geometries_from_place(area, tags={'amenity': 'fast_food'})
fast_food = fast_food.to_crs(epsg)
fast_food = fast_food[['geometry']]
fast_food = fast_food.reset_index()

Daily Shops

In [42]:
#Extracting daily shops
daily_shops = ox.geometries_from_place(area, tags={'shop': ['department_store', 'supermarket', 'convenience']})
daily_shops = daily_shops.to_crs(epsg)
daily_shops = daily_shops[['geometry']]
daily_shops = daily_shops.reset_index()

Business Shops

In [53]:
# Extracting business shops
business_shops = ox.geometries_from_place(area, tags={'shop': ['clothes', 'jewelry', 'shoes', 'tailor', 'beauty', 'cosmetics', 'hairdresser',
                                                    'doityourself', 'garden_center', 'hardware', 'mall', 'department_store']})
business_shops = business_shops.to_crs(epsg)
business_shops = business_shops[['geometry']]
business_shops = business_shops.reset_index()

Greenspace

In [65]:
# Extracting greenspace
greenspace = ox.geometries_from_place(area, tags={'leisure': ['garden', 'nature_reserve', 'park', 'pitch']})
greenspace = greenspace.to_crs(epsg)
greenspace = greenspace[['geometry']]
greenspace = greenspace.reset_index()

Water Bodies

In [11]:
# Extracting water bodies
water_bodies = ox.geometries_from_place(area, tags={'water': ['lake', 'river', 'canal', 'rapids', 'lagoon']})
water_bodies = water_bodies.to_crs(epsg)
water_bodies = water_bodies[['geometry']]
water_bodies = water_bodies.reset_index()

Gyms

In [24]:
# Extracting gyms
gym = ox.geometries_from_place(area, tags={'leisure': 'fitness_centre'})
gym = gym.to_crs(epsg)
gym = gym[['geometry']]
gym = gym.reset_index()

Sport Fields

In [ ]:
# Extracting sport fields
sport_fields = ox.geometries_from_place(area, tags={'sport': ['soccer', 'tennis', 'athletics,', 'baseball', 'basketball', 'field_hockey', 'handball',
                                                                        'ice_hockey', 'cricket', 'rugby_league', 'rugby_union', 'softball', 'volleyball']})
sport_fields = sport_fields.to_crs(epsg)
sport_fields = sport_fields[['geometry']]
sport_fields = sport_fields.reset_index()

Schools

In [27]:
# Extracting schools
schools = ox.geometries_from_place(area, tags={'amenity': 'school'})
schools = schools.to_crs(epsg)
schools = schools[['geometry']]
schools = schools.reset_index()

College / Universities

In [40]:
# Extracting college / universities
coluni = ox.geometries_from_place(area, tags={'amenity': ['college', 'university']})
coluni = coluni.to_crs(epsg)
coluni = coluni[['geometry']]
coluni = coluni.reset_index()

Places of Worship

In [8]:
# Extracting places of worship (e.g., churches, mosques, temples, etc)
place_of_worship = ox.geometries_from_place(area, tags={'amenity': 'place_of_worship'})
place_of_worship = place_of_worship.to_crs(epsg)
place_of_worship = place_of_worship[['geometry']]
place_of_worship = place_of_worship.reset_index()

Hospitals

In [6]:
# Extracting hospitals
hospitals = ox.geometries_from_place(area, tags={'amenity': 'hospital'})
hospitals = hospitals.to_crs(epsg)
hospitals = hospitals[['geometry']]
hospitals = hospitals.reset_index()

Doctors

In [14]:
# Extracting doctor's offices
doctor_offices = ox.geometries_from_place(area, tags={'amenity': 'doctors'})
doctor_offices = doctor_offices.to_crs(epsg)
doctor_offices = doctor_offices[['geometry']]
doctor_offices = doctor_offices.reset_index()

Pharmacies

In [16]:
# Extracting pharmacies
pharmacies = ox.geometries_from_place(area, tags={'amenity': 'pharmacy'})
pharmacies = pharmacies.to_crs(epsg)
pharmacies = pharmacies[['geometry']]
pharmacies = pharmacies.reset_index()

Gambling

In [17]:
# Extracting places for gambling, adult gaming centers, and casino's
gambling = ox.geometries_from_place(area, tags={'amenity': ['gambling', 'adult_gaming_centre', 'casino']})
gambling = gambling.to_crs(epsg)
gambling = gambling[['geometry']]
gambling = gambling.reset_index()

### Calculate the nearest node to each POI, allowing distance calculations between grid centroids and POI using the street network

In [14]:
def calculate_distances(facility_df, facility_name):
    def calculate_shortest_path(row):
        try:
            return nx.shortest_path_length(Gproj, valid_nodes_grid[row], valid_nodes[row], weight='length')
        except (nx.NetworkXNoPath, nx.NodeNotFound):
            return None

    def get_nearest_polygon(row):
        return facility_df.distance(row, align=True).sort_values().index[0]

    polygon_index = Parallel(n_jobs=2, prefer='threads')(delayed(get_nearest_polygon)(row)
                                                             for row in popgrid['geometry'][0:1])
    polygon_index = Parallel(n_jobs=-1, prefer='threads')(delayed(get_nearest_polygon)(row) 
                                                              for row in popgrid['geometry'])

    nearest_facility = facility_df.loc[polygon_index].geometry.centroid

    nearest_nodes, dist = ox.distance.nearest_nodes(Gproj, nearest_facility.geometry.x,
                                                    nearest_facility.geometry.y, return_dist=True)

    mask = np.vectorize(isinstance)(nearest_nodes, int)
    nearest_nodes_b = np.where(mask, nearest_nodes, np.nan)
    dist_b = np.where(mask, dist, np.nan)
    valid_nodes = np.array([nearest_nodes_b, dist_b], np.float64)
    valid_nodes = np.where(valid_nodes[1]<1000, valid_nodes[0], np.nan)

    popgrid[f'dist_to_{facility_name}'] = [calculate_shortest_path(x) for x in popgrid.index]

# Usage:
calculate_distances(bus_stops, 'bus_stops')
calculate_distances(train_stations, 'train_stations')
calculate_distances(restaurants, 'restaurants')
calculate_distances(fast_food, 'fast_food')
calculate_distances(daily_shops, 'daily_shops')
calculate_distances(business_shops, 'business_shops')
calculate_distances(greenspace, 'greenspace')
calculate_distances(water_bodies, 'water_bodies')
calculate_distances(gym, 'gym')
calculate_distances(sport_fields, 'sport_fields')
calculate_distances(schools, 'schools')
calculate_distances(coluni, 'coluni')
calculate_distances(place_of_worship, 'place_of_worship')
calculate_distances(hospitals, 'hospitals')
calculate_distances(doctor_offices, 'doctor_offices')
calculate_distances(pharmacies, 'pharmacies')
calculate_distances(gambling, 'gambling')

## Calculate Building Density
- The buffer created in the previous notebook is imported.
- The buildings are extracted from OpenStreetMap (OSMNX).
- The buldings density is calculated for each grid.
- The distribution of the building density is visualized using a density plot.

In [ ]:
# Import the buffer
buffer_gdkg = gpd.read_file('bufferintersect_gdf.gpkg')

In [ ]:
# Extracting buildings from OSMNX
buildings = ox.geometries_from_place(area, tags={'building': True})
buildings = buildings.to_crs(epsg)
buildings = buildings.reset_index()

In [ ]:
# Initialize an empty list to store the calculated build density
building_densities = []

In [ ]:
for index, row in popgrid.iterrows():
    # Find the corresponding buffer polygon from the buffer geopackage
    buffer_polygon = buffer_gdkg.iloc[index]['geometry']

    # Clip the buildings data to the buffer polygon
    buildings_within_buffer = buildings[buildings.intersects(buffer_polygon)]

    # Count the number of buildings within the buffer
    building_count = len(buildings_within_buffer)

    # Calculate the building density as the number of buildings divided by the buffer area
    building_density = building_count / buffer_polygon.area

    # Append the building density to the list
    building_densities.append(building_density)

In [ ]:
# Save diversity_scores as column in dataframe
popgrid['building_density'] = building_densities

In [ ]:
# Convert the building density to per km^2
popgrid['building_density'] = popgrid['building_density']*1000000

In [ ]:
# Visualize the distribution of the building density through a density plot
sns.kdeplot(popgrid['building_density'])
plt.xlabel('Building Density')
plt.ylabel('Density')
plt.title('Density Distribution of Building Density')
plt.show()

## Calculate Land Diversity
- The land use types are extracted from OpenStreetMap (OSMNX).
- The land diversity is calculated for each grid.
- The distribution of the land diversity is visualized using a density plot.

In [ ]:
# Extracting land use from OSMNX
land_use = ox.geometries_from_place(area, tags={'landuse': True})
land_use = land_use.to_crs(epsg)
land_use = land_use.reset_index()

In [ ]:
# Initialize an empty list to store the calculated diversity scores
diversity_scores = []

In [ ]:
for index, row in popgrid.iterrows():
    # Find the corresponding buffer polygon from the buffer geopackage
    buffer_polygon = buffer_gdkg.iloc[index]['geometry']

    # Intersect the land use data with the buffer polygon
    land_use_within_buffer = land_use[land_use.intersects(buffer_polygon)]

    # Calculate the frequency of each land use category within the buffer
    land_use_counts = land_use_within_buffer['landuse'].value_counts()

    # Calculate the probability of each land use category
    land_use_probs = land_use_counts / land_use_counts.sum()

    # Calculate the Shannon diversity score
    diversity_score = -np.sum(land_use_probs * np.log(land_use_probs))

    # Append the diversity score to the list
    diversity_scores.append(diversity_score)

In [ ]:
# Save diversity_scores as column in dataframe
popgrid['land_diversity'] = diversity_scores

In [ ]:
# Visualize the distribution of the land use diversity through a density plot
sns.kdeplot(popgrid['land_diversity'])
plt.xlabel('Diversity Score')
plt.ylabel('Density')
plt.title('Density Distribution of Land Use Diversity Scores')
plt.show()

## Analyzing and exporting the output
- The head and tail of the finilized dataframe is return to manually inspect the extracted and calculated variables.
- Additionally, the columns types are return for manually inspection.
- The finilized dataframe is exported as a GeoPackage (GPKG).

In [18]:
popgrid

,level_0,index,dissolve_key,row3,col3,population,grid_lon,grid_lat,grid_lon_4326,grid_lat_4326,...,dist_to_place_of_worship,dist_to_sport_fields,dist_to_hospitals,dist_to_fast_food,dist_to_water_bodies,geometry,center,dist_to_doctor_offices,dist_to_pharmacies,dist_to_gambling
0,0,0,10-621,10,621,0,390427.904831,420645.529660,-2.146435,53.682200,...,4621.159,5797.481,6023.933,5154.379,7924.513,"POLYGON ((390417.254 420647.897, 390433.850 42...","(53.68220010828345, -2.1464348064045695)",9425.060,6851.492,18271.813
1,1,1,10-628,10,628,0,390871.732026,420671.933032,-2.139770,53.682474,...,4621.159,5797.481,6023.933,5154.379,7924.513,"POLYGON ((390837.929 420707.144, 390893.725 42...","(53.682474316822464, -2.139769932112907)",9425.060,6851.492,18271.813
2,2,2,10-629,10,629,0,390925.124111,420675.744940,-2.138872,53.682475,...,4344.859,5521.181,5747.633,4878.079,7648.213,"POLYGON ((390893.725 420707.144, 390956.523 42...","(53.68247535958708, -2.1388721697636233)",9148.760,6575.192,17995.513
3,3,3,10-630,10,630,0,390987.922453,420675.744940,-2.137921,53.682476,...,4344.859,5521.181,5747.633,4878.079,7648.213,"POLYGON ((390956.523 420707.144, 391019.322 42...","(53.682476456797296, -2.1379213962979837)",9148.760,6575.192,17995.513
4,4,4,10-631,10,631,0,391050.720795,420675.744940,-2.136971,53.682478,...,4344.859,5521.181,5747.633,4878.079,7648.213,"POLYGON ((391019.322 420707.144, 391082.120 42...","(53.68247754645913, -2.136970622769079)",9148.760,6575.192,17995.513
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
324747,324747,324747,99-740,99,740,0,397895.740085,415086.692492,-2.033295,53.632316,...,7288.524,4525.565,9232.092,7403.964,5628.122,"POLYGON ((397864.341 415118.092, 397927.139 41...","(53.63231598685202, -2.0332945497947104)",6694.766,6916.713,13031.342
324748,324748,324748,99-741,99,741,0,397958.538427,415086.692492,-2.032345,53.632316,...,7288.524,4525.565,9232.092,7403.964,5628.122,"POLYGON ((397927.139 415118.092, 397989.938 41...","(53.63231623603037, -2.032344900788797)",6694.766,6916.713,13031.342
324749,324749,324749,99-742,99,742,0,398021.336769,415086.692492,-2.031395,53.632316,...,7288.524,4525.565,9232.092,7403.964,5628.122,"POLYGON ((397989.938 415118.092, 398052.736 41...","(53.63231647767457, -2.031395251768364)",6694.766,6916.713,13031.342
324750,324750,324750,99-743,99,743,0,398084.135111,415086.692492,-2.030446,53.632317,...,7288.524,4525.565,9232.092,7403.964,5628.122,"POLYGON ((398052.736 415118.092, 398115.534 41...","(53.632316711784654, -2.0304456027338516)",6694.766,6916.713,13031.342


In [19]:
popgrid.dtypes

level_0                        int64
index                          int64
dissolve_key                  object
row3                           int64
col3                           int64
population                     int64
grid_lon                     float64
grid_lat                     float64
grid_lon_4326                float64
grid_lat_4326                float64
buffer_population            float64
buffer_area                  float64
buffer_pop_density           float64
dist_to_restaurant           float64
dist_to_bus_stop             float64
dist_to_daily_shops          float64
dist_to_business_shops       float64
dist_to_greenspace           float64
dist_to_train_stations       float64
dist_to_gym                  float64
dist_to_schools              float64
dist_to_coluni               float64
dist_to_place_of_worship     float64
dist_to_sport_fields         float64
dist_to_hospitals            float64
dist_to_fast_food            float64
dist_to_water_bodies         float64
g

In [21]:
gdf.to_file('popgridmanchesternew.gpkg', driver='GPKG')